In [1]:
# importing important Libraries
import pandas as pd

In [2]:
df = pd.read_csv("/home/omkar/Omkar Pawar/Data Science/Projects/Diamonds Price Predictions/artifacts/raw.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
 10  price    193573 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 16.2+ MB


In [4]:
back_up_df = df # Keeping the original data frame just in case we need it in future.
df = df.sample(n=15000, random_state=47)
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 80639 to 152846
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       15000 non-null  int64  
 1   carat    15000 non-null  float64
 2   cut      15000 non-null  object 
 3   color    15000 non-null  object 
 4   clarity  15000 non-null  object 
 5   depth    15000 non-null  float64
 6   table    15000 non-null  float64
 7   x        15000 non-null  float64
 8   y        15000 non-null  float64
 9   z        15000 non-null  float64
 10  price    15000 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 1.4+ MB


In [5]:
df = df.drop(labels=['id'], axis=1)

* separating dependent and independent variables:

In [6]:
X = df.drop(labels=['price'], axis= 1)
Y = df[['price']]

In [7]:
Y

,price
80639,743
180511,2564
163826,2364
146615,1963
74483,10872
...,...
98785,693
177564,1668
187476,1343
119917,3002


In [8]:
X

,carat,cut,color,clarity,depth,table,x,y,z
80639,0.33,Premium,E,SI1,62.6,59.0,4.40,4.45,2.77
180511,0.72,Premium,I,VS1,61.1,56.0,5.82,5.77,3.54
163826,0.72,Ideal,F,SI2,61.3,56.0,5.76,5.74,3.52
146615,0.56,Ideal,D,VS2,60.7,56.0,5.33,5.37,3.25
74483,1.70,Ideal,I,VS2,61.2,55.0,7.67,7.62,4.66
...,...,...,...,...,...,...,...,...,...
98785,0.32,Ideal,E,VS1,61.2,55.0,4.43,4.47,2.72
177564,0.52,Ideal,F,VS1,61.9,55.0,5.17,5.14,3.20
187476,0.53,Good,E,SI1,63.8,56.0,5.14,5.18,3.29
119917,0.73,Premium,G,VS1,62.3,59.0,5.76,5.79,3.59


In [9]:
# Defining which columns should be ordinal-encoded and which should be scaled

categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [10]:
# Defining the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [11]:
from sklearn.impute import SimpleImputer # HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [12]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])


In [13]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [14]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [15]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-1.042043,1.758071,-1.164932,-1.300139,-1.285315,-1.166377,-2.141882,0.915104,-0.658484
1,0.657494,1.009603,1.964455,0.761121,0.734729,0.860918,-0.128655,0.915104,-0.658484
2,-0.611780,-0.113099,-1.164932,-0.490037,-0.515343,-0.510059,0.877959,0.294755,0.685100
3,-0.848425,0.167577,-1.686496,-0.868085,-0.850507,-0.845511,0.877959,0.294755,-0.658484
4,-1.063556,0.074018,0.921326,-1.327143,-1.312491,-1.312226,-1.135268,1.535454,-0.658484


In [16]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [27]:
fitted = regression.predict(X_test)
fitted

array([[15178.78433115],
       [ 1499.79118878],
       [ 1157.20512833],
       ...,
       [  268.74250141],
       [ 4235.86028238],
       [ 6152.86012726]])

In [28]:

residuls = y_test - fitted
residuls # Error 

,price
174471,2880.215669
17195,644.208811
97646,-43.205128
39206,-93.884422
180364,19.071604
...,...
66025,1497.707410
9981,-470.831097
186673,255.257499
101833,261.139718


In [18]:
regression.coef_

array([[ 6527.18877869,    96.60898451,  -111.96162619,     7.98775168,
          433.90577032, -2745.48134971,    52.43620521,  -465.47081617,
          668.26878096]])

In [19]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [20]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 997.1868836811158
MAE: 679.7015197870845
R2 score 93.78417612248438




Lasso
Model Training Performance
RMSE: 999.6240786590236
MAE: 681.6687212259529
R2 score 93.75375516967989


Ridge
Model Training Performance
RMSE: 997.7537710555158
MAE: 680.368993835836
R2 score 93.7771068885915


Elasticnet
Model Training Performance
RMSE: 1528.9883451275043
MAE: 1080.185413916947
R2 score 85.38651037860797




/home/omkar/Omkar Pawar/Data Science/Projects/Diamonds Price Predictions/dpp_venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.058e+07, tolerance: 1.769e+07
  model = cd_fast.enet_coordinate_descent(


In [21]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']